In [5]:
'''
迴圈讀取本地列表，
    上傳設定檔，取得id，並將id寫入檔案中，而後上傳圖片

'''

import json
from linebot import LineBotApi

# 載入安全設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))



'''
rich_menu的本地列表
'''
rich_menu_array=['detect_menu','main_menu','story_menu']

from linebot.models import RichMenu

for rich_menu_name in rich_menu_array:

    
    # 創建菜單，取得menuId
    lineRichMenuId = line_bot_api.create_rich_menu(rich_menu=RichMenu.new_from_json_dict(json.load(open("./richmenu/"+rich_menu_name+'/rich_menu.json','r'))))
    print("-設定檔上傳結果")
    print(lineRichMenuId)

                                           
    # id寫入本地端                              
    f = open("richmenu/"+rich_menu_name+"/rich_menu_id", "w")
    f.write(lineRichMenuId)
    f.close()                                       
 

    # 上傳照片至該id
    set_image_response=''
    with open("richmenu/"+rich_menu_name+'/rich_menu.jpg', 'rb') as f:
        set_image_response=line_bot_api.set_rich_menu_image(lineRichMenuId, 'image/jpeg', f)
        
    print("-圖片上傳結果")                                               
    print(set_image_response)
                                                                 


-設定檔上傳結果
richmenu-21037e40acaa34b5d8014e70ac9e6f7c
-圖片上傳結果
None
-設定檔上傳結果
richmenu-717ccefd3df008bba45c121fe447371b
-圖片上傳結果
None
-設定檔上傳結果
richmenu-fae403e7211b5143ee4e7e5be0ceabd1
-圖片上傳結果
None


In [6]:
'''

綁定個人用戶進行測試

'''

from linebot import (
    LineBotApi
)

import json

# rich_menu_id_array = ["rich_menu_0"]

secretFileContentJson=json.load(open("./line_secret_key",'r'))
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))

self_user_id='U27915f51b4978ddee0207fac63f46b9d'
rich_menu_id='richmenu-a956e2156d42da96605fe62d81ec912f'

line_bot_api.unlink_rich_menu_from_user(self_user_id)
line_bot_api.link_rich_menu_to_user(self_user_id,rich_menu_id)

LineBotApiError: LineBotApiError: status_code=404, error_response={"details": [], "message": "richmenu not found"}

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'


In [7]:
'''

查詢帳號內擁有的richmenu 

'''

from linebot import (
    LineBotApi
)

import json

secretFileContentJson=json.load(open("./line_secret_key",'r'))
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))

line_bot_api.get_rich_menu_list()


[{"areas": [{"action": {"data": "model=yolov3_tree", "type": "postback"}, "bounds": {"height": 383, "width": 798, "x": 23, "y": 23}}, {"action": {"data": "model=yolov3_leaf", "type": "postback"}, "bounds": {"height": 373, "width": 779, "x": 863, "y": 28}}, {"action": {"data": "model=clr_tree", "type": "postback"}, "bounds": {"height": 368, "width": 779, "x": 42, "y": 448}}, {"action": {"data": "model=clr_leaf", "type": "postback"}, "bounds": {"height": 369, "width": 788, "x": 858, "y": 443}}, {"action": {"data": "menu=main_menu", "type": "postback"}, "bounds": {"height": 369, "width": 774, "x": 1684, "y": 28}}, {"action": {"data": "menu=story_menu", "type": "postback"}, "bounds": {"height": 359, "width": 779, "x": 1684, "y": 448}}], "chatBarText": "\u597d\u304d\u306a\u30e2\u30c7\u30eb\u3092\u304a\u9078\u3073\u304f\u3060\u3055\u3044", "name": "AI\u6a21\u578b\u9078\u55ae", "richMenuId": "richmenu-17edf5a8631f3225392a1f8dc9493c4f", "selected": true, "size": {"height": 843, "width": 2500}}

In [ ]:
'''

移除帳號內的richmenu

'''

from linebot import (
    LineBotApi
)

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))

# 設定要移除的rich_menu
rich_menu_name_array = ['richmenu_05']

for rich_menu_name in rich_menu_name_array:
    
    # 讀取rich_menu_id檔案，並告知 Line 進行刪除，並在刪除後，把本地檔案內容清除
    with open("richmenu/"+rich_menu_name+'/rich_menu_id', 'r') as myfile:
        rich_menu_id = myfile.read()
        deleteResult = line_bot_api.delete_rich_menu(rich_menu_id)
        print(deleteResult)
        
    f = open("richmenu/"+rich_menu_name+"/rich_menu_id", "w")
    f.write('')
    f.close() 

In [4]:
'''

解除用戶綁定

'''

from linebot import (
    LineBotApi
)

import json

# rich_menu_id_array = ["rich_menu_0"]

secretFileContentJson=json.load(open("./line_secret_key",'r'))
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))

self_user_id='U2e9d01c4f9377230375f9c6b1908bb87'
line_bot_api.unlink_rich_menu_from_user(self_user_id)